In [2]:
from mip import Model, xsum, maximize, BINARY, minimize, INTEGER

In [ ]:
m = Model("fradir")

#decision variable
deltaH = [m.add_var(var_type=INTEGER) for l in all_link]
Z = [m.add_var(var_type=BINARY) for s in all_S]
Y = [m.add_var(var_type=BINARY) for l in all_link]

#objective
m.objective = minimize(xsum(c[l] * deltaH[l] for l in all_link))

#cons1
m += xsum(1 - ((Hnull[l] + deltaH[l]) / I) for l in all_link) <= Y[l]

#cons2
for s in all_S:
    m += xsum(y[l]  for l in s) - len(s) + 1 <= Z[s]

#cons3
for s in all_S:
    m += xsum(Pr[p,m] * Z[s, p, m] for p in P for m in M) <= T

m.optimize()


# ---------------------------------------------------------------

In [25]:
#koltseg =  [11, 3,3, 6, 1, 9, 16,78]
koltseg = 8
fejlesztes = [17,67,4,61,16,42,156, 7]

y = [1,0,1,1,0,1,0,0]
z = [0,1,1,1,0,1,0,1]

items = len(fejlesztes)

m = Model("fradir")

#decision variable
x = [m.add_var(var_type=BINARY) for i in range(items)]
fejlesztes_Dvar = [m.add_var(var_type=INTEGER, lb=1, ub=6) for i in range(items)]

#objective
m.objective = minimize(xsum(koltseg * fejlesztes[i] * x[i] for i in range(items)))

#cons
m += xsum(y[i] * x[i] for i in range(items)) == 1

m += xsum(z[i] * x[i] for i in range(items)) == 1

#m += xsum((y[i] + z[i]) * x[i] for i in range(items)) == 2

#m += xsum(x[i] for i in range(items)) == 1


m.optimize()

selected = [i for i in range(items) if x[i].x >= 0.99]
print("selected items: {}".format(selected))

selected items: [2]


# GEKKO

In [26]:
import random
from gekko import GEKKO
import numpy as np

In [27]:
koltseg =  [11, 3,3, 6, 1, 9, 16,78]
fejlesztes = [17,67,4,61,16,42,156, 7]

y = [1,0,1,1,0,1,1,0]
z = [0,1,1,1,0,1,1,1]

prob = [0.5, 1.2, 1.5, 1.7, 0.1, 1.8, 0.4, 1.9]

items = len(koltseg)

# Create model
m = GEKKO()

# Variables
x = m.Array(m.Var,items,lb=0,ub=1,integer=True)
fejlesztes_Dvar = m.Array(m.Var, items, lb=1,ub=6, integer=True)
#x2 = m.Array(m.Var, len(w),lb=0,ub=1,integer = True)

# Objective
m.Maximize(sum(koltseg[i] * fejlesztes_Dvar[i] * x[i] for i in range(items) ))

# Constraint
m.Equation(sum([(y[i] + z[i]) * x[i] for i in range(items)]) == 2)

m.Equation(sum([x[i] for i in range(items)]) == 1)

m.Equation(sum((z[i] * prob[i]) *x[i] for i in range(items)) <=1.6 )

# Optimize with APOPT
m.options.SOLVER = 1

m.solve(disp = False)

# Print the value of the variables at the optimum
print(x)
for i in range(len(x)):
    if x[i][0] == 1.0:
        print("Sorszam: {}., koltseg: {}, fejlesztes: {}".format(i,koltseg[i], fejlesztes_Dvar[i][0]))

[[0.0] [0.0] [0.0] [0.0] [0.0] [0.0] [1.0] [0.0]]
Sorszam: 6., koltseg: 16, fejlesztes: 6.0
